In [75]:
import pandas as pd
import numpy as np
import json
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import classification_report
import pickle

In [76]:
df_EUROSTAT = pd.read_csv("EUROSTAT/data/serie/serie_EU.csv")
df_GTREND = pd.read_csv('GTrends/data/final/gtrend_monthly_2017.csv')
df_GDELT = pd.read_csv('GDELT/data/final/gdelt_monthly.csv')
    
data = pd.concat([df_GTREND, df_GDELT], axis=1)
data.head()
    

,Date,Media,Media_Education_busqueda_mensual,Media_Investments_busqueda_mensual,Media_Jobs_busqueda_mensual,Media_Mortage_busqueda_mensual,Media_Unemployement_benefits_busqueda_mensual,Media_Unemployments_busqueda_mensual,Media_Welfare_busqueda_mensual,Media_Investments_busqueda_mensual.1,...,Media_Volume_intensity_Financial_inclusion,Media_Volume_intensity_Health_Service,Media_Volume_intensity_Inclusive_Growth,Media_Volume_intensity_Social_inclusion,Media_average_tone_Education,Media_Average_tone_Enterprise_Development,Media_Average_tone_Financial_Inclusion,Media_Average_tone_Health_Service,Media_Average_tone_Inclusive_Growth,Media_Average_tone_Social_inclusion
0,2017-01-31,34.526316,34.526316,31.052632,70.631579,5.368421,7.263158,45.000000,0.000000,31.052632,...,0.000400,0.032663,0.001526,0.016526,-4.047779,3.672095,0.955005,-3.134963,0.792732,0.280137
1,2017-02-28,34.000000,34.000000,33.236842,68.526316,10.052632,11.263158,34.157895,0.000000,33.236842,...,0.000300,0.029216,0.000989,0.013747,-4.251689,4.221805,3.418921,-3.644742,3.066589,0.669737
2,2017-03-31,36.210526,36.210526,34.131579,66.421053,15.368421,8.631579,45.631579,4.333333,34.131579,...,0.000495,0.037516,0.001384,0.019537,-9.263563,5.326442,5.087337,-10.663121,7.076711,3.958305
3,2017-04-30,30.894737,30.894737,31.500000,60.157895,9.000000,6.052632,39.473684,0.000000,31.500000,...,0.000542,0.033011,0.001689,0.017421,-1.940011,3.883058,3.712237,-4.871816,2.563179,2.432684
4,2017-05-31,33.263158,33.263158,31.868421,61.578947,5.157895,3.105263,41.578947,0.000000,31.868421,...,0.000563,0.051711,0.007711,0.018816,-2.871253,4.308063,0.567563,-22.223179,1.172721,5.180226


In [77]:
data.columns

Index(['Date', 'Media', 'Media_Education_busqueda_mensual',
       'Media_Investments_busqueda_mensual', 'Media_Jobs_busqueda_mensual',
       'Media_Mortage_busqueda_mensual',
       'Media_Unemployement_benefits_busqueda_mensual',
       'Media_Unemployments_busqueda_mensual',
       'Media_Welfare_busqueda_mensual',
       'Media_Investments_busqueda_mensual.1', 'Media_Jobs_busqueda_mensual.1',
       'Media_Mortage_busqueda_mensual.1',
       'Media_Unemployement_benefits_busqueda_mensual.1',
       'Media_Unemployments_busqueda_mensual.1',
       'Media_Welfare_busqueda_mensual.1', 'Date',
       'Media_volumen_intensity_Education',
       'Media_Volume_intensity_Enterprise',
       'Media_Volume_intensity_Financial_inclusion',
       'Media_Volume_intensity_Health_Service',
       'Media_Volume_intensity_Inclusive_Growth',
       'Media_Volume_intensity_Social_inclusion',
       'Media_average_tone_Education',
       'Media_Average_tone_Enterprise_Development',
       'Media_Aver

In [78]:
data.drop(columns=['Media'],axis=1, inplace=True) # eliminamos media y date (que sale duplicada)

In [79]:
data

,Date,Media_Education_busqueda_mensual,Media_Investments_busqueda_mensual,Media_Jobs_busqueda_mensual,Media_Mortage_busqueda_mensual,Media_Unemployement_benefits_busqueda_mensual,Media_Unemployments_busqueda_mensual,Media_Welfare_busqueda_mensual,Media_Investments_busqueda_mensual.1,Media_Jobs_busqueda_mensual.1,...,Media_Volume_intensity_Financial_inclusion,Media_Volume_intensity_Health_Service,Media_Volume_intensity_Inclusive_Growth,Media_Volume_intensity_Social_inclusion,Media_average_tone_Education,Media_Average_tone_Enterprise_Development,Media_Average_tone_Financial_Inclusion,Media_Average_tone_Health_Service,Media_Average_tone_Inclusive_Growth,Media_Average_tone_Social_inclusion
0,2017-01-31,34.526316,31.052632,70.631579,5.368421,7.263158,45.000000,0.000000,31.052632,70.631579,...,0.000400,0.032663,0.001526,0.016526,-4.047779,3.672095,0.955005,-3.134963,0.792732,0.280137
1,2017-02-28,34.000000,33.236842,68.526316,10.052632,11.263158,34.157895,0.000000,33.236842,68.526316,...,0.000300,0.029216,0.000989,0.013747,-4.251689,4.221805,3.418921,-3.644742,3.066589,0.669737
2,2017-03-31,36.210526,34.131579,66.421053,15.368421,8.631579,45.631579,4.333333,34.131579,66.421053,...,0.000495,0.037516,0.001384,0.019537,-9.263563,5.326442,5.087337,-10.663121,7.076711,3.958305
3,2017-04-30,30.894737,31.500000,60.157895,9.000000,6.052632,39.473684,0.000000,31.500000,60.157895,...,0.000542,0.033011,0.001689,0.017421,-1.940011,3.883058,3.712237,-4.871816,2.563179,2.432684
4,2017-05-31,33.263158,31.868421,61.578947,5.157895,3.105263,41.578947,0.000000,31.868421,61.578947,...,0.000563,0.051711,0.007711,0.018816,-2.871253,4.308063,0.567563,-22.223179,1.172721,5.180226
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,2023-08-31,33.842105,35.657895,69.368421,27.368421,4.736842,34.736842,6.400000,35.657895,69.368421,...,0.000505,0.036768,0.000895,0.026026,2.699911,0.972653,1.639584,-2.553526,1.921889,3.748800
80,2023-09-30,81.210526,38.947368,73.578947,22.000000,17.947368,38.421053,11.200000,38.947368,73.578947,...,0.000600,0.053237,0.001153,0.032379,1.873968,1.947374,2.527058,-2.279511,4.693637,2.865300
81,2023-10-31,69.684211,40.052632,71.052632,16.368421,14.947368,36.315789,0.000000,40.052632,71.052632,...,0.000958,0.067684,0.000800,0.029032,0.847111,2.966500,4.077995,-3.398916,3.051705,5.063089
82,2023-11-30,46.947368,40.184211,66.526316,28.473684,19.105263,41.000000,0.000000,40.184211,66.526316,...,0.000505,0.054689,0.000542,0.031379,-0.045589,0.764821,1.736558,-3.381432,0.606300,6.923858


In [80]:
df_EUROSTAT

,year_month,total
0,2017-03-01,212800.8
1,2017-06-01,215840.8
2,2017-09-01,219059.4
3,2017-12-01,220487.7
4,2018-03-01,221984.1
5,2018-06-01,226911.9
6,2018-09-01,227300.1
7,2018-12-01,228923.4
8,2019-03-01,229417.1
9,2019-06-01,233126.9


In [81]:
df_EUROSTAT.index

RangeIndex(start=0, stop=28, step=1)

In [82]:
# Cambiamos el nombre de la columna "year_month" a "Date", para que se llame como las de X
df_EUROSTAT.rename(columns={'year_month': 'Date'}, inplace=True)
df_EUROSTAT['Date'] = pd.to_datetime(df_EUROSTAT['Date'])
df_EUROSTAT.set_index('Date', inplace=True)

# Rellenamos los valores faltantes de los meses anteriores con el mes siguiente
df_EUROSTAT_RES = df_EUROSTAT.resample('M').ffill()
df_EUROSTAT_RES
# Plantearnos si es mejor el bfill hasta enero en lugar de ir para abajo, pues de las otras series tenemos datos
# hasta enero

,total
Date,
2017-03-31,212800.8
2017-04-30,212800.8
2017-05-31,212800.8
2017-06-30,215840.8
2017-07-31,215840.8
...,...
2023-08-31,267535.4
2023-09-30,268847.7
2023-10-31,268847.7


In [85]:
# Prueba rick
X = data.set_index("Date")
X

,Media_Education_busqueda_mensual,Media_Investments_busqueda_mensual,Media_Jobs_busqueda_mensual,Media_Mortage_busqueda_mensual,Media_Unemployement_benefits_busqueda_mensual,Media_Unemployments_busqueda_mensual,Media_Welfare_busqueda_mensual,Media_Investments_busqueda_mensual.1,Media_Jobs_busqueda_mensual.1,Media_Mortage_busqueda_mensual.1,...,Media_Volume_intensity_Financial_inclusion,Media_Volume_intensity_Health_Service,Media_Volume_intensity_Inclusive_Growth,Media_Volume_intensity_Social_inclusion,Media_average_tone_Education,Media_Average_tone_Enterprise_Development,Media_Average_tone_Financial_Inclusion,Media_Average_tone_Health_Service,Media_Average_tone_Inclusive_Growth,Media_Average_tone_Social_inclusion
Date,,,,,,,,,,,,,,,,,,,,,
"(2017-01-31, 2017-01-31 00:00:00+00:00)",34.526316,31.052632,70.631579,5.368421,7.263158,45.000000,0.000000,31.052632,70.631579,5.368421,...,0.000400,0.032663,0.001526,0.016526,-4.047779,3.672095,0.955005,-3.134963,0.792732,0.280137
"(2017-02-28, 2017-02-28 00:00:00+00:00)",34.000000,33.236842,68.526316,10.052632,11.263158,34.157895,0.000000,33.236842,68.526316,10.052632,...,0.000300,0.029216,0.000989,0.013747,-4.251689,4.221805,3.418921,-3.644742,3.066589,0.669737
"(2017-03-31, 2017-03-31 00:00:00+00:00)",36.210526,34.131579,66.421053,15.368421,8.631579,45.631579,4.333333,34.131579,66.421053,15.368421,...,0.000495,0.037516,0.001384,0.019537,-9.263563,5.326442,5.087337,-10.663121,7.076711,3.958305
"(2017-04-30, 2017-04-30 00:00:00+00:00)",30.894737,31.500000,60.157895,9.000000,6.052632,39.473684,0.000000,31.500000,60.157895,9.000000,...,0.000542,0.033011,0.001689,0.017421,-1.940011,3.883058,3.712237,-4.871816,2.563179,2.432684
"(2017-05-31, 2017-05-31 00:00:00+00:00)",33.263158,31.868421,61.578947,5.157895,3.105263,41.578947,0.000000,31.868421,61.578947,5.157895,...,0.000563,0.051711,0.007711,0.018816,-2.871253,4.308063,0.567563,-22.223179,1.172721,5.180226
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(2023-08-31, 2023-08-31 00:00:00+00:00)",33.842105,35.657895,69.368421,27.368421,4.736842,34.736842,6.400000,35.657895,69.368421,27.368421,...,0.000505,0.036768,0.000895,0.026026,2.699911,0.972653,1.639584,-2.553526,1.921889,3.748800
"(2023-09-30, 2023-09-30 00:00:00+00:00)",81.210526,38.947368,73.578947,22.000000,17.947368,38.421053,11.200000,38.947368,73.578947,22.000000,...,0.000600,0.053237,0.001153,0.032379,1.873968,1.947374,2.527058,-2.279511,4.693637,2.865300
"(2023-10-31, 2023-10-31 00:00:00+00:00)",69.684211,40.052632,71.052632,16.368421,14.947368,36.315789,0.000000,40.052632,71.052632,16.368421,...,0.000958,0.067684,0.000800,0.029032,0.847111,2.966500,4.077995,-3.398916,3.051705,5.063089


In [88]:
index_original = X.index # para poder cogerlo luego para el que formemos
# Reorganizar X para tener 3 columnas por serie (t-0, t-1, t-2)
n_rows = len(X) - 2  # número de filas después de la reorganización
num_series = len(X.columns)  # número de series en X
columns = [f't-{i}' for i in range(num_series * 3)]  # nombres de las nuevas columnas

X_transformed = pd.DataFrame(index=range(n_rows), columns=columns)

for i in range(n_rows):
    for j in range(num_series):
        X_transformed.loc[i, [f't-{j*3}', f't-{j*3+1}', f't-{j*3+2}']] = [X.iloc[i, j], X.iloc[i+1, j], X.iloc[i+2, j]]

X_transformed
# pero aqui estan todas juntas?? no hay diferencias? es lo que tiene que ser? y no tenemos fecha 

,t-0,t-1,t-2,t-3,t-4,t-5,t-6,t-7,t-8,t-9,...,t-65,t-66,t-67,t-68,t-69,t-70,t-71,t-72,t-73,t-74
0,34.526316,34.0,36.210526,31.052632,33.236842,34.131579,70.631579,68.526316,66.421053,5.368421,...,5.087337,-3.134963,-3.644742,-10.663121,0.792732,3.066589,7.076711,0.280137,0.669737,3.958305
1,34.0,36.210526,30.894737,33.236842,34.131579,31.5,68.526316,66.421053,60.157895,10.052632,...,3.712237,-3.644742,-10.663121,-4.871816,3.066589,7.076711,2.563179,0.669737,3.958305,2.432684
2,36.210526,30.894737,33.263158,34.131579,31.5,31.868421,66.421053,60.157895,61.578947,15.368421,...,0.567563,-10.663121,-4.871816,-22.223179,7.076711,2.563179,1.172721,3.958305,2.432684,5.180226
3,30.894737,33.263158,28.736842,31.5,31.868421,32.052632,60.157895,61.578947,60.315789,9.0,...,3.459447,-4.871816,-22.223179,-13.186921,2.563179,1.172721,7.629805,2.432684,5.180226,3.728142
4,33.263158,28.736842,24.105263,31.868421,32.052632,27.868421,61.578947,60.315789,62.421053,5.157895,...,5.422105,-22.223179,-13.186921,-6.504005,1.172721,7.629805,2.446695,5.180226,3.728142,1.634263
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,34.578947,31.842105,33.842105,37.894737,35.657895,35.657895,65.421053,65.578947,69.368421,20.947368,...,1.639584,2.365763,0.473663,-2.553526,1.972342,0.509674,1.921889,7.142853,6.737316,3.7488
78,31.842105,33.842105,81.210526,35.657895,35.657895,38.947368,65.578947,69.368421,73.578947,19.473684,...,2.527058,0.473663,-2.553526,-2.279511,0.509674,1.921889,4.693637,6.737316,3.7488,2.8653
79,33.842105,81.210526,69.684211,35.657895,38.947368,40.052632,69.368421,73.578947,71.052632,27.368421,...,4.077995,-2.553526,-2.279511,-3.398916,1.921889,4.693637,3.051705,3.7488,2.8653,5.063089
80,81.210526,69.684211,46.947368,38.947368,40.052632,40.184211,73.578947,71.052632,66.526316,22.0,...,1.736558,-2.279511,-3.398916,-3.381432,4.693637,3.051705,0.6063,2.8653,5.063089,6.923858


In [98]:
X_transformed = X_transformed.values

In [99]:
y = df_EUROSTAT_RES.values

In [100]:
# PRUEBA LSTM, X, CON SELECTKBEST, PARA REDUCIR DIMENSIONALIDAD Y TENER DIMENSIONES ADECUADAS PARA EL MODELO
X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=0.2, shuffle=False)
lag_steps = 3
n_features = 25,39 # NO ENCAJA CON LA DIMENSIONALIDAD!!

# Reformateamos los datos para que sean 3D (número de muestras, número de pasos de tiempo, número de características)
X_train = X_train.reshape(X_train.shape[0], lag_steps, n_features)
X_test = X_test.reshape(X_test.shape[0], lag_steps, n_features)

model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(lag_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

model.fit(X_train, y_train, epochs=20, batch_size=8, validation_data=(X_test, y_test))
loss = model.evaluate(X_test, y_test)
print("Pérdida en conjunto de prueba:", loss)
predictions = model.predict(X_test)

ValueError: cannot reshape array of size 4875 into shape (65,3,9)

In [102]:
# Vamos a intentarlo con Random Forest, usamos X normal
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
# primero, escalamos
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(X_transformed)
X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=0.2, shuffle=False)
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)
rf_regressor.fit(X_train, y_train)
predictions = rf_regressor.predict(X_test)
mse = mean_squared_error(y_test, predictions)
print("Error cuadrático medio:", mse)
rmse = np.sqrt(mse)
print("Root Mean Squared Error (RMSE):", rmse)
r2 = r2_score(y_test, predictions)
print("Coeficiente de determinación (R^2):", r2)
# el error es muy alto, Y EL COEFICIENTE DE DETERMINACION TOTALMENTE NEGATIVO. 
# es igual que en la otra prueba

c:\Users\Picar\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Error cuadrático medio: 1603482315.7822237
Root Mean Squared Error (RMSE): 40043.50528840131
Coeficiente de determinación (R^2): -46.19095066640365


In [103]:
# CON XGBOOST
import xgboost as xgb
from sklearn.metrics import mean_squared_error
X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=0.2, shuffle=False)
xgb_regressor = xgb.XGBRegressor(objective ='reg:squarederror', n_estimators=100, random_state=42)
xgb_regressor.fit(X_train, y_train)
predictions = xgb_regressor.predict(X_test)
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
print("Root Mean Squared Error (RMSE):", rmse)
r2 = r2_score(y_test, predictions)
print("Coeficiente de determinación (R^2):", r2)
# Igual que en el otro, el error es muy alto, y sobre todo el r cuadrado.

Root Mean Squared Error (RMSE): 33408.32376522899
Coeficiente de determinación (R^2): -31.847621170355836
